EMEA Contracts

In [0]:
"""
silver_emea_cleaning.py

Silver layer cleaning and transformation script for EMEA vendor contracts.
This script loads the bronze Delta table, applies normalization and aggregation,
pads missing fields to match the unified schema, and prepares the data for consolidation.
"""

import re
from pyspark.sql.functions import (
    col, upper, trim, when, lit, current_timestamp, to_date,
    sum, min, max, count
)

# 📥 Load bronze table
bronze_df = spark.table("contracts_pipeline.bronze_emea_contracts")

# 🧼 Clean column names: replace non-word characters with underscores
for col_name in bronze_df.columns:
    bronze_df = bronze_df.withColumnRenamed(col_name, re.sub(r"[^\w]", "_", col_name))

# 🔢 Standardize Contract_Number and remove invalid entries
bronze_df = bronze_df.withColumn(
    "Contract_Number",
    when(trim(upper(col("Contract_Number"))).isin("N/A", ""), None)
    .otherwise(trim(upper(col("Contract_Number"))))
)

# 🧹 Drop rows with missing Contract_Number
bronze_df = bronze_df.filter(col("Contract_Number").isNotNull())

# 🧾 Drop unnecessary columns
columns_to_drop = [col_name for col_name in bronze_df.columns if "Funding_Cost_Center" in col_name]
bronze_df = bronze_df.drop(*columns_to_drop)

# 🔄 Convert FY26 to numeric
bronze_df = bronze_df.withColumn("FY26", col("FY26").cast("double"))

# 📅 Convert date columns
bronze_df = bronze_df.withColumn("Start_Date", to_date(col("Start_Date"))) \
                     .withColumn("End_Date", to_date(col("End_Date")))

# 🏷️ Add metadata
bronze_df = bronze_df.withColumn("import_timestamp", current_timestamp()) \
                     .withColumn("contract_region", lit("EMEA")) \
                     .withColumn("source_table", lit("contracts_pipeline.bronze_emea_contracts"))

# 📊 Aggregate annualized cost and resource count
agg_df = bronze_df.groupBy("Contract_Number", "Vendor", "GL__Global_Finance_").agg(
    sum("FY26").alias("annualized_cost"),
    min("Start_Date").alias("Start"),
    max("End_Date").alias("End"),
    count("GL__Global_Finance_").alias("resource_count")
).withColumn("import_timestamp", current_timestamp()) \
 .withColumn("contract_region", lit("EMEA")) \
 .withColumn("source_table", lit("contracts_pipeline.bronze_emea_contracts")) \
 .withColumnRenamed("GL__Global_Finance_", "Goods_Type")

# 🧩 Pad missing columns to match unified silver schema
emea_silver = agg_df.select(
    col("Contract_Number"),
    lit(None).cast("string").alias("contract_name"),
    col("Vendor"),
    col("Start"),
    col("End"),
    lit(None).cast("string").alias("Description"),
    lit(None).cast("string").alias("Renewal_Strategy"),
    lit(None).cast("string").alias("Expiration"),
    lit(None).cast("string").alias("Latest_Comment"),
    lit(None).cast("string").alias("BuyingHub"),
    lit(None).cast("date").alias("TPMO_Meeting_Date"),
    lit(None).cast("date").alias("CPMO_Meeting_Date"),
    lit(None).cast("string").alias("Renewal_BuyingHub"),
    lit(None).cast("string").alias("Contract_Type"),
    lit(None).cast("double").alias("Contract_Value"),
    lit(None).cast("double").alias("Total_Contract_Value"),
    lit(None).cast("string").alias("Domain_Owner"),
    lit(None).cast("string").alias("Pillar"),
    lit(None).cast("string").alias("Sub_Status"),
    col("Goods_Type"),
    lit(None).cast("string").alias("Funding_Source"),
    lit(None).cast("double").alias("FY25_Committed"),
    lit(None).cast("string").alias("FY25_Committed_Display"),
    lit(None).cast("double").alias("FY26_Committed"),
    lit(None).cast("string").alias("FY26_Committed_Display"),
    col("annualized_cost").alias("Annualized_Cost"),
    lit(None).cast("string").alias("Sub_Domain"),
    lit(None).cast("string").alias("Leader_Owner"),
    lit(None).cast("int").alias("days_from_end_date"),
    lit(None).cast("boolean").alias("next_41_days"),
    lit(None).cast("string").alias("Renewal_Contract_Number"),
    lit(None).cast("string").alias("Renewal_Status"),
    lit(None).cast("double").alias("Renewal_Contract_Value"),
    lit(None).cast("date").alias("Renewal_CPMO_Meeting_Date"),
    lit(None).cast("date").alias("Renewal_TPMO_Meeting_Date"),
    lit(None).cast("string").alias("Divest_Label"),
    lit(None).cast("string").alias("Original_Governance_Required"),
    lit(None).cast("string").alias("Governance_Required"),
    col("import_timestamp")
)

# 🧪 Create temp view for downstream union
emea_silver.createOrReplaceTempView("emea_vendor_contracts_silver_temp")


Apla data

In [0]:
"""
silver_apla_cleaning.py

Silver layer cleaning and transformation script for APLA vendor contracts.
This script pulls raw contract data from Smartsheet, applies normalization and aggregation,
pads missing fields to match the unified schema, and prepares the data for consolidation.
"""

# Install required package
%pip install smartsheet_dataframe

import pandas as pd
from smartsheet_dataframe import get_sheet_as_df
from pyspark.sql.functions import (
    col, lit, sum, min, max, count, first
)

# 🔐 Replace with your own Smartsheet token or use a secure secret manager
smartsheet_token = "your-smartsheet-token"

# 📥 Retrieve data from Smartsheet using the sheet ID
sheet_id = "your-sheet-id"
df = get_sheet_as_df(token=smartsheet_token, sheet_id=sheet_id)

# 🧹 Drop unwanted columns
df.drop(columns=[col for col in df.columns if col in ["Funding_Cost_Center", "Muge Present"]], inplace=True, errors="ignore")

# 🧾 Rename columns to align with global schema
df.rename(columns={
    "Detailed Description": "Description",
    "Plan at Contract Expiration": "Renewal_Strategy",
    "Expired Contract Note": "Latest_Comment"
}, inplace=True)

# 🕒 Add import timestamp
df["import_timestamp"] = pd.Timestamp.now()

# 🔢 Normalize numeric columns
numeric_columns = [
    "Previous Contract Value", "Total Contract Value", "FY25 (Committed)", "FY26 (Committed)",
    "Annualized Cost", "PO/NExT", "Contract Length (months)", "Days from End Date",
    "Helper Exp Value", "Renewal Total Value", "Original - Renewal Total Value"
]
for column in numeric_columns:
    if column in df.columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")

# 🔤 Convert object-type columns to strings
for col_name in df.select_dtypes(include="object").columns:
    df[col_name] = df[col_name].astype(str)

# 🧾 Normalize key string columns
string_columns = ["Cost Center", "Contract #", "BuyingHub", "Vendor"]
for column in string_columns:
    if column in df.columns:
        df[column] = df[column].astype(str)

# 🧼 Clean column names
df.columns = df.columns.str.replace(r"[^\w]", "_", regex=True)
df.rename(columns={"Contract__": "Contract_Number"}, inplace=True)

# 🔄 Convert to Spark DataFrame
df_spark = spark.createDataFrame(df)

# 🏷️ Add metadata columns
df_spark = df_spark.withColumn("contract_region", lit("APLA")) \
                   .withColumn("source_table", lit("Smartsheet: apla_vendor_contracts"))

# 📊 Aggregate by Contract_Number
agg_df = df_spark.groupBy("Contract_Number").agg(
    sum("Annualized_Cost").alias("annualized_cost"),
    sum("Total_Contract_Value").alias("Total_Contract_Value"),
    first("Vendor").alias("Vendor"),
    first("Description").alias("Description"),
    first("Renewal_Strategy").alias("Renewal_Strategy"),
    first("Latest_Comment").alias("Latest_Comment"),
    min("Start").alias("Start"),
    max("End").alias("End"),
    min("import_timestamp").alias("import_timestamp"),
    count("*").alias("resource_count")
)

# 🧩 Pad missing columns to match unified silver schema
apla_silver = agg_df.select(
    col("Contract_Number"),
    lit(None).cast("string").alias("contract_name"),
    col("Vendor"),
    col("Start"),
    col("End"),
    col("Description"),
    col("Renewal_Strategy"),
    lit(None).cast("string").alias("Expiration"),
    col("Latest_Comment"),
    lit(None).cast("string").alias("BuyingHub"),
    lit(None).cast("date").alias("TPMO_Meeting_Date"),
    lit(None).cast("date").alias("CPMO_Meeting_Date"),
    lit(None).cast("string").alias("Renewal_BuyingHub"),
    lit(None).cast("string").alias("Contract_Type"),
    lit(None).cast("double").alias("Contract_Value"),
    col("Total_Contract_Value"),
    lit(None).cast("string").alias("Domain_Owner"),
    lit(None).cast("string").alias("Pillar"),
    lit(None).cast("string").alias("Sub_Status"),
    lit(None).cast("string").alias("Goods_Type"),
    lit(None).cast("string").alias("Funding_Source"),
    lit(None).cast("double").alias("FY25_Committed"),
    lit(None).cast("string").alias("FY25_Committed_Display"),
    lit(None).cast("double").alias("FY26_Committed"),
    lit(None).cast("string").alias("FY26_Committed_Display"),
    col("annualized_cost").alias("Annualized_Cost"),
    lit(None).cast("string").alias("Sub_Domain"),
    lit(None).cast("string").alias("Leader_Owner"),
    lit(None).cast("int").alias("days_from_end_date"),
    lit(None).cast("boolean").alias("next_41_days"),
    lit(None).cast("string").alias("Renewal_Contract_Number"),
    lit(None).cast("string").alias("Renewal_Status"),
    lit(None).cast("double").alias("Renewal_Contract_Value"),
    lit(None).cast("date").alias("Renewal_CPMO_Meeting_Date"),
    lit(None).cast("date").alias("Renewal_TPMO_Meeting_Date"),
    lit(None).cast("string").alias("Divest_Label"),
    lit(None).cast("string").alias("Original_Governance_Required"),
    lit(None).cast("string").alias("Governance_Required"),
    col("import_timestamp")
)

# 🧪 Create temp view for downstream union
apla_silver.createOrReplaceTempView("apla_vendor_contracts_silver_temp")

# 👀 Display for verification
display(spark.sql("SELECT * FROM apla_vendor_contracts_silver_temp"))


Global Data

In [0]:
-- silver_global_contracts.sql

-- Silver layer transformation for global vendor contracts.
-- This query powers a dashboard view that links expiring contracts to their renewal counterparts,
-- enabling side-by-side comparison of contract terms, values, governance classification, and expiration risk.

-- Step 1: Create permanent silver table
CREATE OR REPLACE TABLE contracts_pipeline.silver_global_contracts AS
WITH ContractData AS (
    SELECT 
        c1.Contract_Number AS Contract_Number,
        c1.Contract_Name AS contract_name,
        c1.Vendor,
        c1.Start,
        c1.End,
        c1.Description,
        c1.Renewal_Strategy,
        c1.Expiration,
        c1.VP_Reviewed,
        c1.Latest_Comment,
        c1.BuyingHub,
        c1.TPMO_Meeting_Date,
        c1.CPMO_Meeting_Date,
        c1.Renewal_BuyingHub,
        c1.Contract_Type,
        c1.Total_Contract_Value AS Contract_Value,
        FORMAT_NUMBER(c1.Total_Contract_Value, 2) AS Total_Contract_Value,
        c1.Domain_Owner,
        c1.Pillar,
        c1.Sub_Status,
        c1.Goods_Type,
        c1.Funding_Source,
        c1.FY25_Committed,
        FORMAT_NUMBER(COALESCE(c1.FY25_Committed, 0), 2) AS FY25_Committed_Display,
        c1.FY26_Committed,
        FORMAT_NUMBER(COALESCE(c1.FY26_Committed, 0), 2) AS FY26_Committed_Display,
        c1.Annualized_Cost,
        c1.Sub_Domain,
        c1.Leader_Owner,

        -- 🧠 Expiration risk classification
        CASE 
            WHEN c1.Expiration LIKE '%New%' THEN 'New'
            WHEN c1.End IS NULL THEN 'No End Date'
            WHEN c1.End < CURRENT_DATE THEN 'Expired'
            WHEN DATEDIFF(c1.End, CURRENT_DATE) <= 30 THEN 'Next 30 days'
            WHEN DATEDIFF(c1.End, CURRENT_DATE) <= 60 THEN 'Next 60 days'
            WHEN DATEDIFF(c1.End, CURRENT_DATE) <= 90 THEN 'Next 90 days'
            WHEN DATEDIFF(c1.End, CURRENT_DATE) <= 180 THEN 'Next 180 days'
            ELSE 'Beyond 180 days'
        END AS days_from_end_date,

        -- 🔔 Flag contracts expiring soon
        CASE 
            WHEN DATEDIFF(c1.End, CURRENT_DATE) <= 41 THEN 'Expires in Next 41 Days'
            ELSE NULL 
        END AS next_41_days,

        -- 🔄 Renewal contract linkage
        COALESCE(c2.Contract_Number, 'N/A') AS Renewal_Contract_Number,
        COALESCE(c2.Sub_Status, 'N/A') AS Renewal_Status,
        COALESCE(FORMAT_NUMBER(c2.Total_Contract_Value, 2), 'N/A') AS Renewal_Contract_Value,
        COALESCE(STRING(c2.CPMO_Meeting_Date), 'N/A') AS Renewal_CPMO_Meeting_Date,
        COALESCE(STRING(c2.TPMO_Meeting_Date), 'N/A') AS Renewal_TPMO_Meeting_Date,

        -- 🏷️ Governance classification
        CASE 
            WHEN c1.Expiration LIKE '%Divest%' THEN 'Divest'  
            ELSE NULL 
        END AS Divest_Label,

        CASE 
            WHEN c1.Total_Contract_Value > 1000000 AND c1.Total_Contract_Value <= 5000000 THEN 'TPMO'
            WHEN c1.Total_Contract_Value > 5000000 THEN 'CPMO'
            ELSE 'No'
        END AS Original_Governance_Required,

        CASE 
            WHEN c2.Total_Contract_Value <= 1000000 THEN 'No'
            WHEN c2.Total_Contract_Value > 1000000 AND c2.Total_Contract_Value < 5000000 THEN 'TPMO'
            WHEN c2.Total_Contract_Value >= 5000000 THEN 'CPMO'
            ELSE 'No'
        END AS Governance_Required,

        -- 🧹 Deduplication: keep latest version per contract
        ROW_NUMBER() OVER (
          PARTITION BY c1.Contract_Number 
          ORDER BY c1.End DESC
        ) AS rn

    FROM contracts_pipeline.bronze_global_contracts c1
    LEFT JOIN (
        SELECT 
            Contract_Number, 
            BuyingHub, 
            Sub_Status, 
            Total_Contract_Value, 
            MAX(End) AS Latest_End,
            CPMO_Meeting_Date,
            TPMO_Meeting_Date
        FROM contracts_pipeline.bronze_global_contracts
        GROUP BY 
            Contract_Number, 
            BuyingHub, 
            Sub_Status, 
            Total_Contract_Value,
            CPMO_Meeting_Date,
            TPMO_Meeting_Date
    ) c2 
    ON c1.Renewal_BuyingHub = c2.BuyingHub
    WHERE c1.Expiration NOT LIKE '%Expired%'  
)
SELECT * 
FROM ContractData
WHERE rn = 1  
ORDER BY End;

-- Step 2: Display the newly created silver table
SELECT * 
FROM contracts_pipeline.silver_global_contracts
ORDER BY End;


Silver Table

In [0]:
-- silver_consolidation.sql

-- Final silver layer consolidation query.
-- Combines cleaned contract data from all regional silver layers (Global, EMEA, APLA)
-- into a unified dataset for dashboarding and analytics.

-- Each source contributes standardized fields, and a `source_label` is added for traceability.

SELECT * FROM (

  -- 🌍 Global contracts (formerly Dave's data)
  SELECT 
    Contract_Number,
    contract_name,
    Vendor,
    Start,
    End,
    Description,
    Renewal_Strategy,
    Expiration,
    Latest_Comment,
    BuyingHub,
    TPMO_Meeting_Date,
    CPMO_Meeting_Date,
    Renewal_BuyingHub,
    Contract_Type,
    Contract_Value,
    Total_Contract_Value,
    Domain_Owner,
    Pillar,
    Sub_Status,
    Goods_Type,
    Funding_Source,
    FY25_Committed,
    FY25_Committed_Display,
    FY26_Committed,
    FY26_Committed_Display,
    Annualized_Cost,
    Sub_Domain,
    Leader_Owner,
    days_from_end_date,
    CAST(next_41_days AS STRING) AS next_41_days,
    Renewal_Contract_Number,
    Renewal_Status,
    Renewal_Contract_Value,
    Renewal_CPMO_Meeting_Date,
    Renewal_TPMO_Meeting_Date,
    Divest_Label,
    Original_Governance_Required,
    Governance_Required,
    import_timestamp,
    'Global' AS source_label
  FROM contracts_pipeline.silver_global_contracts

  UNION ALL

  -- 🌍 EMEA contracts
  SELECT 
    Contract_Number,
    contract_name,
    Vendor,
    Start,
    End,
    Description,
    Renewal_Strategy,
    Expiration,
    Latest_Comment,
    BuyingHub,
    TPMO_Meeting_Date,
    CPMO_Meeting_Date,
    Renewal_BuyingHub,
    Contract_Type,
    Contract_Value,
    Total_Contract_Value,
    Domain_Owner,
    Pillar,
    Sub_Status,
    Goods_Type,
    Funding_Source,
    FY25_Committed,
    FY25_Committed_Display,
    FY26_Committed,
    FY26_Committed_Display,
    Annualized_Cost,
    Sub_Domain,
    Leader_Owner,
    days_from_end_date,
    CAST(next_41_days AS STRING) AS next_41_days,
    Renewal_Contract_Number,
    Renewal_Status,
    Renewal_Contract_Value,
    Renewal_CPMO_Meeting_Date,
    Renewal_TPMO_Meeting_Date,
    Divest_Label,
    Original_Governance_Required,
    Governance_Required,
    import_timestamp,
    'EMEA' AS source_label
  FROM emea_vendor_contracts_silver_temp

  UNION ALL

  -- 🌍 APLA contracts (Territory)
  SELECT 
    Contract_Number,
    contract_name,
    Vendor,
    Start,
    End,
    Description,
    Renewal_Strategy,
    Expiration,
    Latest_Comment,
    BuyingHub,
    TPMO_Meeting_Date,
    CPMO_Meeting_Date,
    Renewal_BuyingHub,
    Contract_Type,
    Contract_Value,
    Total_Contract_Value,
    Domain_Owner,
    Pillar,
    Sub_Status,
    Goods_Type,
    Funding_Source,
    FY25_Committed,
    FY25_Committed_Display,
    FY26_Committed,
    FY26_Committed_Display,
    Annualized_Cost,
    Sub_Domain,
    Leader_Owner,
    days_from_end_date,
    CAST(next_41_days AS STRING) AS next_41_days,
    Renewal_Contract_Number,
    Renewal_Status,
    Renewal_Contract_Value,
    Renewal_CPMO_Meeting_Date,
    Renewal_TPMO_Meeting_Date,
    Divest_Label,
    Original_Governance_Required,
    Governance_Required,
    import_timestamp,
    'APLA' AS source_label
  FROM territory_vendor_contracts_silver_temp

);
